In [ ]:
# inspired by fawazsammani


import numpy as np

import os

import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm



In [ ]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    def __len__(self):
        return len(self.word2idx)
        

In [ ]:
class TextProcess():
    
    def __init__(self):
        self.dictionary = Dictionary()



    def get_data(self, path, batch_size=20):

        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)  
        
        rep_tensor = torch.LongTensor(tokens)  # index of words in the file
        
        
        index = 0

        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2idx[word]
                    index += 1


        num_batches = rep_tensor.shape[0] // batch_size     
        rep_tensor = rep_tensor[:num_batches*batch_size]
        
        rep_tensor = rep_tensor.view(batch_size, -1)  # (batch_size, num_batches)
        
        
        return rep_tensor



In [ ]:
embed_size = 128
hidden_size = 1024
num_layers = 1

batch_size = 20

num_epochs = 20
timesteps = 30

learning_rate = 0.002

In [ ]:
corpus = TextProcess()

In [ ]:
rep_tensor = corpus.get_data('alice.txt', batch_size)

In [ ]:
print(rep_tensor.shape)  # each row contains 1659 words by default

In [ ]:
vocab_size = len(corpus.dictionary)
print(vocab_size)

In [ ]:
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

In [ ]:
class TextGenerator(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()

        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)



    def forward(self, x, h):
        
        x = self.embed(x)
        # x = x.view(batch_size, timesteps, embed_size)

        out, (h, c) = self.lstm(x, h)

        # out = (samples, timesteps, output_features) --> (batch_size*timesteps, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        out = self.linear(out)


        return out, (h, c)

In [ ]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def detach(states):

    return [state.detach() for state in states]


In [ ]:
for epoch in range(num_epochs):

    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))


    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):

        inputs = rep_tensor[:, i:i+timesteps]  
        targets = rep_tensor[:, (i+1):(i+1)+timesteps]
        
        outputs,_ = model(inputs, states)
        loss = loss_fn(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()

        clip_grad_norm(model.parameters(), 0.5)  # maximum allowed value of the gradients clipped in
                                                 # the range [-clip_value, clip_value]
        optimizer.step()

        
              
        step = (i+1) // timesteps
        
        if step % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:

with torch.no_grad():

    with open('results.txt', 'w') as f:

        state = (torch.zeros(num_layers, 1, hidden_size),
                 torch.zeros(num_layers, 1, hidden_size))

        input = torch.randint(0,vocab_size, (1,)).long().unsqueeze(1)  # one word id randomly, shape (1, 1)


        for i in range(500):

            output, _ = model(input, state)
            print(output.shape)

            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            print(word_id)

            input.fill_(word_id)


            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)
            
            
            if (i+1) % 100 == 0:
                print(f'Sampled [{i+1}/{500}] words and save to {"results.txt"}')
